In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

In [8]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib

In [161]:
import requests 
from bs4 import BeautifulSoup 

In [162]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
contents = page.content


In [163]:
soup = BeautifulSoup(contents, 'lxml') #used BeautifulSoup to retrieve the data form wikipedia
links =soup.find_all("a")


In [164]:
# by using soup i selected all the tables and chose the first table (the one that i need)

a = soup.find_all("table")[0] 
data = pd.read_html(str(a))
df = data[0]

print(df.shape)
print(df.head())


(288, 3)
  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront


In [165]:
# i got rid of the rows where Borough = Not assigned 

df=df[df.Borough != 'Not assigned']
df = df.sort_values(by=['Postcode','Borough'])

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

print(df.shape)
print(df.head())

(211, 3)
  Postcode      Borough   Neighbourhood
0      M1B  Scarborough           Rouge
1      M1B  Scarborough         Malvern
2      M1C  Scarborough  Highland Creek
3      M1C  Scarborough      Rouge Hill
4      M1C  Scarborough      Port Union


In [179]:
# here i combined the Neighborhoods with the same Postal

dfNew = df.groupby(['Postcode', 'Borough'], sort=False).agg( ','.join)
dfNew = dfNew.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)
dfNew = dfNew.sample(frac=1).reset_index()
dfNew['Neighbourhood']= dfNew['Neighbourhood'].str.join(', ')


In [177]:
# here i replace the Neighbourhood = Not assigned with corresponding Borough value

dfNew['Neighbourhood'].loc[dfNew['Neighbourhood'] == 'Not assigned'] = dfNew['Borough']


In [180]:
dfNew.shape

(103, 3)